# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [ ]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time

The line below will retrieve your shared variables from the first notebook.

In [ ]:
%store -r

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [ ]:
forecast = boto3.client(service_name='forecast') 
forecastquery = boto3.client(service_name='forecastquery')

## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
predictorName= project+'_deeparp_algo'

In [ ]:
forecastHorizon = 24

In [ ]:
algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [ ]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [ ]:
predictor_arn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

### Get Error Metrics

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=predictor_arn)

## Create a Forecast

Now create a forecast using the model that was trained

In [ ]:
forecastName= project+'_deeparp_algo_forecast'

In [ ]:
create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
while True:
    forecastStatus = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    print(forecastStatus)
    if forecastStatus != 'ACTIVE' and forecastStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

### Get Forecast

Once created, the forecast results are ready and you view them. 

In [ ]:
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_12"}
)
print(forecastResponse)

## Next Steps

Now that your forecast has been created, to evaluate it use `3.Evaluating_Your_Predictor.ipynb` but before opening it, execute the cell below to share your variables again with the next notebook.

In [ ]:
%store forecast_arn
%store predictor_arn